In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/vertex_endpoints/find_ideal_machine_type/find_ideal_machine_type.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/vertex_endpoints/find_ideal_machine_type/find_ideal_machine_type.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/vertex_endpoints/find_ideal_machine_type/find_ideal_machine_type.ipynb"
      <img src="https://cloud.google.com/images/products/ai/ai-solutions-icon.svg" alt="Vertex AI Workbench notebook"> Open in Vertex AI Workbench
    </a>
  </td> 
</table>

# Determining the ideal machine type to use for Vertex AI endpoints

## Overview
This tutorial demonstrates how to determine the ideal machine type for your machine learning model based on cost and performance requirements.

More details about best practices can be found [here](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute#finding_the_ideal_machine_type)

## Model
The model used for this tutorial is `BERT` from [TensorFlow Hub open source model repository](https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4)

## Objective

The steps performed include:
- Create a workbench notebook with a machine type that is being tested.
- Download the model from TensorFlow Hub.
- Create a local model and deploy it to a local endpoint.
- Benchmark the model latencies.
- Clean up.

## Costs
This tutorial uses billable components of Google Cloud:
- Vertex AI
- Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

4. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

5. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you might be able to get your project ID using `gcloud`.

In [ ]:
import os

PROJECT_ID = ""

if not os.getenv("IS_TESTING"):
    # Get your Google Cloud project ID from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = ""  # @param {type:"string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

You first upload the model files to a Cloud Storage bucket. Using this model artifact, you can then
create Vertex AI model and endpoint resources in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with Vertex AI.

In [ ]:
BUCKET_NAME = ""  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

In [ ]:
print(BUCKET_NAME)

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -p $PROJECT_ID -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Create a Workbench Notebook

You'll be using a Google Cloud Notebook to run load tests on a specific machine type to get a good idea of how your model will perform once it is running in a Vertex AI endpoint.

Here we will be creating the notebook using `gcloud`, but you can also create it through the Google cloud console as explained [here](https://cloud.google.com/vertex-ai/docs/workbench/user-managed/create-new#before_you_begin).

In [ ]:
!gcloud notebooks instances create load-test-notebook \
--vm-image-project="deeplearning-platform-release" \
--vm-image-name="common-cpu-notebooks-v20221017-debian-10" \
--machine-type="n1-standard-8" --project=$PROJECT_ID \
--location=us-central1-a

### Open The Workbench Notebook

Once the notebook is created, open the notebook. You'll be running the rest of the steps in the newly created notebook.

### Install Vegeta

Vegeta is a versatile HTTP load testing tool built out of a need to drill HTTP services with a constant request rate.

In [ ]:
! wget https://github.com/tsenart/vegeta/releases/download/v12.8.4/vegeta_12.8.4_linux_amd64.tar.gz

In [ ]:
! tar -xvf vegeta_12.8.4_linux_amd64.tar.gz

### Install dependencies

Install the python dependencies

In [ ]:
%%writefile requirements.txt
google-cloud-aiplatform[prediction]>=1.16.0,<2.0.0
matplotlib
fastapi
contexttimer
tqdm

In [ ]:
%pip install -U --user -r requirements.txt

### Download and extract the model

In [ ]:
! wget https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4?tf-hub-format=compressed -O bert.tgz
! mkdir -p bert_sentence_embedding/00001
! tar -xvf bert.tgz -C bert_sentence_embedding/00001

### Set the bucket variable in the new notebook

You created a bucket in a previous step. Because you are now working on a new notebook, you should re-set the bucket variable

In [ ]:
BUCKET_NAME = ""  # @param {type:"string"}

### Configuration

In order to send requests to the endpoint, you'll create a dummy request body.

In [ ]:
# The gcs uri; remember to have a version folder under this link
# For example, GCS_URI = "gs://project/bucket/folder"
# the model should be put in "gs://project/bucket/folder/1/saved_model.pb".
GCS_URI = f"gs://{BUCKET_NAME}/bert_sentence_embedding"
REQUEST = """
{
  "instances": [
    {
      "input_word_ids": [101, 23784, 11591, 11030, 24340, 21867, 21352, 21455, 20467, 10159, 23804, 10822, 26534, 20355, 14000, 11767, 10131, 28426, 10576, 22469, 22237, 25433, 263, 28636, 12291, 119, 15337, 10171, 25585, 21885, 10263, 13706, 16046, 10112, 18725, 13668, 12208, 10104, 13336, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      "input_mask": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      "input_type_ids": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    }
  ]
}
"""

In [ ]:
!echo $GCS_URI

### Copy model to GCS bucket

In [ ]:
!sudo gsutil cp -r ./bert_sentence_embedding/00001/* $GCS_URI/1/

### Logging
Turn on logging to see the logs of the model

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

## Sequential requests

This tests what the latency (and, potentially, utilization), when the server is serving
at most 1 request at a time, back-to-back. You can use this information to estimate how
many QPS a single replica can handle as a starting point for your configuration.

### Monkey patch LocalModel to provide cleaner syntax...

In [ ]:
from google.cloud.aiplatform.prediction import LocalModel


@classmethod
def create_tensorflow2(
    cls, version: str, saved_model_path: str, includes_version_subdir: bool = True
) -> LocalModel:
    version = version.replace(".", "-")
    return cls(
        serving_container_image_uri=f"us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.{version}:latest",
        serving_container_predict_route="/v1/models/default:predict",
        serving_container_health_route="/v1/models/default",
        serving_container_ports=[8501],
        serving_container_environment_variables={
            "model_name": "default",
            "model_path": saved_model_path,
        },
    )


LocalModel.create_tensorflow2 = create_tensorflow2


@classmethod
def create_pytorch(cls, version: str) -> LocalModel:
    version = version.replace(".", "-")
    return LocalModel(
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/pytorch-gpu.{version}:latest",
        serving_container_predict_route="/predictions/model",
        serving_container_health_route="/ping",
        serving_container_ports=[8080],
    )


LocalModel.create_pytorch = create_pytorch

### Create the LocalModel and deploy it to a LocalEndpoint.

In [ ]:
from google.cloud.aiplatform.prediction import LocalModel

local_model = LocalModel.create_tensorflow2(version="2.7", saved_model_path=GCS_URI)

In [ ]:
import os.path

GPU_COUNT = 1 if os.path.exists("/dev/nvidia0") else None
print(GPU_COUNT)

In [ ]:
from contexttimer import Timer

with Timer() as timer:
    local_endpoint = local_model.deploy_to_local_endpoint(
        gpu_count=GPU_COUNT,
    )
    local_endpoint.serve()

# Actual startup time involves more than just loading the container and model, but still
# a useful number:
print(f"Startup time: {timer.elapsed}")

### Send sequential requests
You'll send multiple requests to the local endpoint and collect latency metrics which will give you a good idea of how the model will perform in production with the selected machine type. You'll visualize these results and get the mean latency in milliseconds. 

Because this is a transformer model, it will run slow on CPUs and would ideally be running using GPUs.

In [ ]:
WARMUP_REQUESTS = 10
NUM_REQUESTS = 100
PERCENTILE_POINTS = [0, 50, 95, 99, 100]
LABELS = ["min", "50", "95", "99", "max"]

import numpy as np
from contexttimer import Timer
from tqdm import tqdm

# Send some warm up requests
for _ in tqdm(range(WARMUP_REQUESTS), desc="Sending warm-up requests"):
    local_endpoint.predict(
        request=REQUEST, headers={"Content-Type": "application/json"}
    )

# Send sequential requests
latencies = []
for _ in tqdm(range(NUM_REQUESTS), desc="Sending requests"):
    with Timer(factor=1000) as timer:
        local_endpoint.predict(
            request=REQUEST, headers={"Content-Type": "application/json"}
        )
    latencies.append(timer.elapsed)

percentiles = np.percentile(latencies, PERCENTILE_POINTS)

In [ ]:
from matplotlib import pyplot as plt

plt.hist(latencies, bins=50, density=True)
plt.xlabel("Latency (ms)")
plt.show()

for p, v in zip(["min", "50", "95", "99", "max"], percentiles):
    print(f"{p}: {v:0.1f}")

print(f"mean: {np.average(latencies):0.1f}")

### Send concurrent requests

The exercise above provides a good benchmark for per request latency, but it is not indicative of how the model will perform in production when there are concurrent requests. For example, latencies might degrade when the machine's resources are exhausted. To find an idea machine type that can handle multiple concurrent requests effectively, we'll use `vegeta`.

In [ ]:
BATCH_SIZE = 1
REQUEST_FILE = "request.json"

import json

instance = json.loads(REQUEST)["instances"][0]
# Row-based encoding
with open(REQUEST_FILE, "w") as f:
    json.dump({"instances": [instance] * BATCH_SIZE}, f)

# Column-based encoding (more efficient for some models)
inputs = {feature: [values] * BATCH_SIZE for feature, values in instance.items()}
with open("request_cols.json", "w") as f:
    json.dump({"inputs": inputs}, f)

In [ ]:
URL = f"http://localhost:{local_endpoint.assigned_host_port}{local_endpoint.serving_container_predict_route}"
URL

In [ ]:
!curl http://localhost:{local_endpoint.assigned_host_port}{local_endpoint.serving_container_health_route}

In [ ]:
!curl -X POST http://localhost:{local_endpoint.assigned_host_port}{local_endpoint.serving_container_predict_route} -d @request.json

In [ ]:
DURATION = "100s"

! for i in 1 2 3 4; do \
    echo "POST {URL}" | \
   ./vegeta attack -header "Content-Type: application/json" -body {REQUEST_FILE} -rate ${{i}} -duration {DURATION} | \
   tee report-${{i}}.bin | \
   ./vegeta report --every=60s; \
  done

In [ ]:
! for f in `ls *.bin`; do \
    ./vegeta report --type=json ${{f}} > ${{f}}.json; \
  done

In [ ]:
import glob
import json
import re

throughput, p99, avg = {}, {}, {}
for fn in glob.glob("report-*.bin.json"):
    with open(fn) as f:
        data = json.load(f)
    qps = int(re.search(r"report-(\d+).bin.json", fn).group(1))
    throughput[qps] = data["throughput"]
    p99[qps] = data["latencies"]["99th"] / 1000000
    avg[qps] = data["latencies"]["mean"] / 1000000

In [ ]:
import matplotlib.pyplot as plt

points = sorted(p99.items(), key=lambda item: item[0])
x, y = zip(*points)
plt.plot(x, y, "-o")
plt.xlabel("Target QPS")
plt.ylabel("P99 Latency (ms)")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

points = sorted(throughput.items(), key=lambda item: item[0])
x, y = zip(*points)
plt.plot(x, y, "-o")
plt.xlabel("Target QPS")
plt.ylabel("Actual QPS")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

points = sorted(avg.items(), key=lambda item: item[0])
x, y = zip(*points)
plt.plot(x, y, "-o")
plt.xlabel("Target QPS")
plt.ylabel("Average Latency (ms)")
plt.show()

We can estimate the number of concurrent requests that a single replica can handle:

$num\_concurrent\_requests = \frac{qps}{avg\_latency_{qps}}$

In [ ]:
QPS = 2

num_concurrent_requests = QPS / avg[QPS]
num_concurrent_requests

#### As you can see, this model would not perform well using this type of machine type. Try different machine type configurations or add a GPU to see how results change.

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
!gsutil rm -r $GCS_URI/*

The following command will delete the workbench notebook instance used for testing. Save all your work before proceeding.

In [ ]:
!gcloud notebooks instances delete load-test-notebook